In [1]:
%matplotlib inline 
import numpy as np
import pandas as pd

import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

from utils import rstr

In [2]:
df =pd.read_csv('data/reviews_metadata.csv',parse_dates=['date'])
df_listing = pd.read_csv('data/listings.csv')

In [3]:
df.date.max()

Timestamp('2017-05-08 00:00:00')

In [4]:
df.head()

,listing_id,date
0,7878281,2015-09-12
1,7878281,2015-10-09
2,7878281,2015-11-15
3,7878281,2016-01-02
4,7878281,2016-04-28


In [5]:
last_scraped=pd.to_datetime(df_listing.last_scraped).max().date()
last_scraped

datetime.date(2017, 5, 8)

In [6]:
df['from_scraping_date']=(last_scraped-df.date).dt.days

In [7]:
bins = [0,7,14,20,30,40,50,60,70,90,100,2000]
group_names = ["reviewed_more_than_"+str(i)+"_ago" for i in bins[:-1]]

In [8]:
categories = pd.cut(df['from_scraping_date'], bins,labels=group_names)
df['review_age']=categories

In [9]:
df.from_scraping_date.head()

0    604
1    577
2    540
3    492
4    375
Name: from_scraping_date, dtype: int64

In [10]:
df_group_review_age = df.groupby(['listing_id','review_age'])['date'].count()
df_group_review_age= df_group_review_age.unstack('review_age').fillna(0)

In [11]:
df_group_review_age.to_csv('data/reviews_frequency.csv')

In [12]:
df_test=pd.read_csv('data/reviews_frequency.csv')

In [13]:
df_listing = df_listing[(df_listing.reviews_per_month>0)&(df_listing.availability_30>0)]

In [14]:
df_test = pd.merge(df_listing[['id','availability_30']],df_test,left_on='id',right_on='listing_id')

In [15]:
df_test.drop(['id','listing_id'],axis=1,inplace=True)

In [19]:
df_test.corr()[['availability_30']].sort_values(by='availability_30',ascending=True)

,availability_30
reviewed_more_than_40_ago,-0.199318
reviewed_more_than_30_ago,-0.193696
reviewed_more_than_20_ago,-0.187662
reviewed_more_than_60_ago,-0.177572
reviewed_more_than_14_ago,-0.168346
reviewed_more_than_70_ago,-0.164634
reviewed_more_than_90_ago,-0.163673
reviewed_more_than_100_ago,-0.142606
reviewed_more_than_50_ago,-0.138040
reviewed_more_than_7_ago,-0.116951
